In [ ]:
# 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #키에 관련된 모듈 가져오기
import time
import sys #파일로 저장하기 위한 모듈
import os #폴더를 다루기 위한 모듈
import pandas as pd #데이터 프레임으로 만들기 위한 모듈
import xlwt #엑셀로 저장을 위해 
import re #정규식 모듈 임포트, 파일명 잘못 생성시 예외처리를 위한 모듈

#브라우저 오픈 함수
def open_br():
    global driver
    global path

    path = 'chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get("https://www.naver.com/")  
    time.sleep(1)
    start()
    
#변수 글로벌 선언 및 변수 초기화, 검색 쿼리 시작 함수    
def start():
    #메뉴 초기화
    global menu_path
    global menu_path2
    
    #파일 저장용 변수 글로벌 선언
    global no
    global no2
    global title2
    global contents2

    global query_txt
    global menu#메뉴 변수 선언    

    #파일 저장용 변수 초기화
    menu_path =''
    menu_path2 = ''

    no= 1
    no2 = []
    title2 = []
    contents2 = []

    query_txt ='' #검색 변수
    menu = 0 #메뉴 변수 선언
   
    
    #검색어 입력
    #검색 쿼리 입력 반복문
    while isNotBlank(query_txt) == False:  #검색어가 비어있다면 반복루프
        query_txt = input('검색어를 입력해 주세요: ') #검색어 변수, 만약 값이 있다면 반복문 탈출이나 빈 문자열이면 반복문 지속
        if isNotBlank(query_txt) == False: #검색어가 비어있다면 해당문구 출력
            print('검색어를 입력하지 않았습니다.')

    print('='*100)
    print('[메뉴] 1.블로그  2.뉴스  3.카페  4.지식백과  5.어학사전 (종료. 0)')
    print()
    
    display_menu()#메뉴 선택 함수 실행
    print('='*100)

    #---------------검색 메뉴 분기 -----------------
    if menu != 0:
        if menu == 1: #블로그
            menu_path = 'VIEW'
            menu_path2 = '블로그'
            print('블로그 섹션의 "%s"(으)로 수집된 데이터를 추출합니다.'%query_txt)
            print()

        elif menu == 2:#뉴스
            menu_path = '뉴스'
            menu_path2 = ''
            print('뉴스 섹션의 "%s"(으)로 수집된 데이터를 추출합니다.'%query_txt)
            print()

        elif menu == 3: #카페
            menu_path = 'VIEW'
            menu_path2 = '카페'
            print('카페 섹션의 "%s"(으)로 수집된 데이터를 추출합니다.'%query_txt)
            print()

        elif menu == 4: #지식백과
            menu_path = '더보기'
            menu_path2 = '지식백과'
            print('지식백과 섹션의 "%s"(으)로 수집된 데이터를 추출합니다.'%query_txt)
            print()

        elif menu == 5: #어학사전
            menu_path = '어학사전'
            menu_path2 = ''
            print('어학사전 섹션의 "%s"(으)로 수집된 데이터를 추출합니다.'%query_txt)
            print()

        #검색 쿼리 실행(검색어, 요소1, 요소2)
        open_query(query_txt, menu_path, menu_path2)

    else :
        close()

        

def open_query(query, menu_path, menu_path2 = ''):

    #검색 페이지에 있는 검색 창에 키워드 입력하고, 검색 버튼 누르기
    element_search = driver.find_element_by_xpath("""//*[@id="query"]""")#검색창 xpath로 찾기
    time.sleep(1)
    element_search.send_keys(query)#검색창에 검색어 입력
    time.sleep(1)
    element_search.send_keys(Keys.ENTER)#검색창에 엔터키 호출

    
    #-----어학사전 존재 체크---------------
    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser')   
    section_list = soup.find_all('section', 'sc_new')
    find_section = [] #어학사전 존재를 찾기 위한 빈 리스트

    for i in section_list:
        try:
            f_menu = i.find('div', 'api_more_wrap').find('a', 'api_more').get_text() #어학사전 더보기를 찾기 위해 크롤링
        except AttributeError :
            continue
        except Exception:
            continue
        else: 
            find_section.append(f_menu) #api more를 find_section 빈 리스트에 담음

    time.sleep(1)
    #-----어학사전 존재 체크 끝---------------

    f_ldc_menu = '어학사전 더보기'
    if (f_ldc_menu in find_section) == False and menu == 5: #find_section에 어학사전 더보기가 없고, 메뉴값이 5(어학사전)이라면
        print('검색된 결과가 없습니다.')
        print()
        time.sleep(1)
        restart()
    else: #어학사전 더보기가 존대하면 기본 크롤링 시작
        time.sleep(1)             

        if menu_path2 == '':
            driver.find_element_by_link_text(menu_path).click()#1차 메뉴 탭 클릭
        else:
            driver.find_element_by_link_text(menu_path).click()#1차 메뉴 탭 클릭
            time.sleep(2)
            driver.find_element_by_link_text(menu_path2).click()#2차 메뉴 탭 클릭  
            
        time.sleep(2)
        
        if menu == 1:#블로그
            func_blog()
        if menu == 2:#뉴스
            func_news()
        if menu == 3:#카페
            func_cafe()
        if menu == 4:#지식백과
            func_dic()
        if menu == 5:#어학사전
            time.sleep(2)#팝업이 열려야함으로 추가 대기
            driver.switch_to_window(driver.window_handles[1])#창전환
            func_ldc()#어학사전 크롤링 함수 실행

            

            
#폴더 생성 및 파일 저장관련 함수
def output_save():
    print('='*100)
    
    #파일저장 선택 예외처리
    save = ['Y', 'y', 'N', 'n']
    ask_save = ""
    while ask_save not in save :
        ask_save = input('추출된 데이터를 파일로 저장하시겠습니까? (Y/N) > ')
        if ask_save in save: 
            continue
        else:
            print('Y, y, N, n 중에서 선택해 주세요.')
    print('='*100)    
    
    currunt_dir = os.getcwd()
    
    #-----카테고리-------
    category = ['블로그', '뉴스', '카페', '지식백과', '어학사전']
    s_category = category[menu - 1]
        
    
    if ask_save == 'Y' or ask_save == 'y':     
        #파일형식선택
        print('[메뉴] 1.txt 2.csv 3.xls 4.전체 저장 (종료: 0)')
        #save_form = 1
        save_menu()
        print('='*100)
        
        #------0번 메뉴 종료 아니면 진행--------
        if save_form == 0:
            close()
            return
        else:             
            #폴더물어보기 및 생성
            #make_dir()
            final_f_dir = ""
            exdir = True #폴더 저장 반복을 위한 변수
            
            while exdir == True: #폴더 저장 반복문 시작
                f_dir = input('''저장할 폴더를 지정해주세요. 현재 프로젝트의 Log 폴더 하위에 생성됩니다. 
(예:today 입력시 -> %s\\today\\ 에 저장)
>'''%currunt_dir)
                
                re_f_dir = re.sub('[\\\/:*?"<>|]','',f_dir)#정규식 금지된 문자열 체크용 변수
                if re_f_dir == '':#금지된 문자열을 다 지웠더니 아무것도 안남았을 때 출력 
                    print("""폴더 생성시 \/:*?"<>| 문자가 포함된 경우 해당 문자가 제거되고, 만약
조정된 폴더명이 빈값이면 기본 폴더에 저장됩니다.""")
                    final_f_dir = currunt_dir+"\\Log\\"              
                elif isNotBlank(f_dir) == False: #폴더명을 아무것도 안적었을 때 조건
                    print('아무 입력값이 없으면 기본설정 폴더에 저장됩니다')
                    final_f_dir = currunt_dir+"\\Log\\"              
                else:  
                    final_f_dir = currunt_dir+"\\Log\{}\\".format(re_f_dir)#정규식으로 처리된 폴더명으로 최종 폴더명 지정
                          
                #폴더 생성 시작
                try:
                    os.makedirs(final_f_dir)
                except FileExistsError:#동일 폴더 생성시 예외처리
                        
                        if re_f_dir !='': #새로 생성한 폴더이름과 동일한 폴더가 있다면 출력
                            print("같은 이름의 폴더가 이미 존재합니다.")
                        d_save = ['Y', 'y', 'N', 'n']
                        d_ask_save = ""
                        while d_ask_save not in d_save :# 'Y', 'y', 'N', 'n' 가 아니라면 반복
                            print('='*100)
                            print(final_f_dir) #최종 폴더 경로 확인용 출력
                            d_ask_save = input('이 폴더에 저장 하시겠습니까?(Y/N)')
                            if d_ask_save in d_save: 
                                continue
                            else:
                                print('Y, y, N, n 중에서 선택해 주세요.')

                        if d_ask_save == 'Y' or d_ask_save == 'y':
                            os.chdir(final_f_dir) #최종 폴더로 이동
                            exdir = False #폴더 생성용 반복문 탈출하기 위해 변수 변경
                        else:
                            continue            

                except OSError:
                    print('파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다')
                else: #폴더 생성이 제대로 되었다면
                    os.chdir(final_f_dir) #최종 폴더로 이동
                    exdir = False #폴더 생성용 반복문 탈출하기 위해 변수 변경

            print('='*100)

            #저장될 파일 위치와 이름을 지정하기
            now = time.localtime()
            #검색어를 생성 금지 문자가 들어갈 경우 정규식을 이용해 처리
            re_query_txt = re.sub('[\\\/:*?"<>|]','',query_txt)
            if re_query_txt == '': #정규식을 통과한 검색어가 통과를 하고 남은 문자열이 없다면
                print(""""파일 생성시 검색어에 \/:*?"<>| 문자가 포함된 경우 해당 문자가 제거되고, 만약
조정된 검색어가 빈값이면 defult로 저장됩니다.""")
                re_query_txt = 'defult' #정규식을 통과한 검색어는 'defult'로 지정
            
            
            f_name = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

            #생성한 폴더로 이동
            ff_name = final_f_dir+f_name+'-'+re_query_txt+'-'+s_category+'.txt'
            fc_name = final_f_dir+f_name+'-'+re_query_txt+'-'+s_category+'.csv'
            fx_name = final_f_dir+f_name+'-'+re_query_txt+'-'+s_category+'.xls'

            naver_view = pd.DataFrame() #네이버 뷰를 데이터프레임 객체로 생성

            #컬럼명 생성뒤 위에서 만든 리스트를 저장
            naver_view['글 번호'] = no2
            naver_view['게시글 제목'] = title2
            naver_view['게시글 요약내용'] = contents2

            time.sleep(2)       

            #---------- 파일 저장 ----------     
            if save_form == 1:
                #txt파일로 저장
                f = open(ff_name, 'a', encoding = 'UTF-8')
                for i in range(0,len(no2)):
                    f.write('글 번호: ' + str(no2[i])+'\n')
                    f.write('게시글 제목: ' + str(title2[i])+'\n')
                    f.write('게시글 요약내용: ' + str(contents2[i])+'\n')
                    f.write('\n')
                f.close()         
                print('{}로 저장되었습니다.'.format(ff_name))
                os.chdir(currunt_dir)
                restart()

            if save_form == 2: 
                #csv 형태로 저장하기
                naver_view.to_csv(fc_name, encoding='utf-8-sig')#지정된 구문 형태      
                print('{}로 저장되었습니다.'.format(fc_name))
                os.chdir(currunt_dir)
                restart()   

            if save_form == 3: 
                #xls 형태로 저장하기
                naver_view.to_excel(fx_name)
                print('{}로 저장되었습니다.'.format(fx_name))
                os.chdir(currunt_dir)
                restart()     

            if save_form == 4:
                #전체 저장

                #txt파일로 저장
                f = open(ff_name, 'a', encoding = 'UTF-8')
                for i in range(0,len(no2)):
                    f.write('글 번호: ' + str(no2[i])+'\n')
                    f.write('게시글 제목: ' + str(title2[i])+'\n')
                    f.write('게시글 요약내용: ' + str(contents2[i])+'\n')
                    f.write('\n')
                f.close()         
                print('{}로 저장되었습니다.'.format(ff_name))        

                #csv 형태로 저장하기
                naver_view.to_csv(fc_name, encoding='utf-8-sig')#지정된 구문 형태      
                print('{}로 저장되었습니다.'.format(fc_name))     

                #xls 형태로 저장하기
                naver_view.to_excel(fx_name)
                print('{}로 저장되었습니다.'.format(fx_name))    

                os.chdir(currunt_dir)
                restart()
        
    else:
        os.chdir(currunt_dir)
        restart()        
        
        

#블로그 섹션 크롤링 함수
def func_blog():
    
    global no
    global no2
    global title2
    global contents2
    
    time.sleep(2)

    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser')   
    
    content_list = soup.find_all("div", attrs={"class":"total_area"})

    print()
    for i in content_list:
        # 번호출력
        no2.append(no)
        print('번호:', no)
        no += 1
        try:
            title = i.find("a", attrs={"class":"api_txt_lines total_tit"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            title2.append(title)
            print('제목:' ,title.strip())
            
        try:
            contents = i.find("div", attrs={"class":"total_group"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            contents2.append(contents)
            print('내용:' ,contents.strip())
            print('\n')
    
    cheak_contents() #크롤링한 데이터가 존재하는지 체크
        
        

#뉴스 섹션 크롤링 함수
def func_news():
    global no
    global no2
    global title2
    global contents2
    
    time.sleep(2)

    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser') 
    
    content_list = soup.find_all("div", attrs={"class":"news_area"})
   
    for i in content_list:
        no2.append(no)
        print('번호:', no)
        no += 1
        try:
            title = i.find("a", attrs={"class":"news_tit"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            title2.append(title)
            print('제목:' ,title.strip())
            
        try:
            contents = i.find("div", attrs={"class":"news_dsc"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            contents2.append(contents)
            print('내용:' ,contents.strip())
            print('\n')
            
    cheak_contents() #크롤링한 데이터가 존재하는지 체크
        
        
#카페 섹션 크롤링 함수
def func_cafe():
    global no
    global no2
    global title2
    global contents2
    
    time.sleep(2)

    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser') 

    content_list = soup.find_all("div", attrs={"class":"total_area"})
    for i in content_list:
        # 번호출력
        no2.append(no)
        print('번호:', no)
        no += 1
        try:
            title = i.find("a", attrs={"class":"api_txt_lines total_tit"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            title2.append(title)
            print('제목:' ,title.strip())
        try:
            contents = i.find("div", attrs={"class":"total_group"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            contents2.append(contents)
            print('내용:' ,contents.strip())
            print('\n')
    
    
    cheak_contents() #크롤링한 데이터가 존재하는지 체크
        
        

#지식백과 섹션 크롤링 함수
def func_dic():
    global no
    global no2
    global title2
    global contents2
    
    time.sleep(2)

    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser') 
    
    content_list = soup.find_all("div", attrs={"class":"nkindic_basic api_ani_send _svp_item"})
    for i in content_list:
        # 번호출력
        no2.append(no)
        print('번호:', no)
        no += 1
        try:
            title = i.find("div", attrs={"class":"nkindic_tit _svp_content"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            title2.append(title)
            print('제목:' ,title.strip())
        try:
            contents = i.find("div", attrs={"class":"api_txt_lines desc"}).get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            contents2.append(contents)
            print('내용:' ,contents.strip())
            print('\n')   
            
    cheak_contents() #크롤링한 데이터가 존재하는지 체크
        
        

#어학사전 섹션 크롤링 함수
#팝업 처리때문에 다른 섹션 함수와 마지막이 살짝 다름
def func_ldc():   
    global no
    global no2
    global title2
    global contents2

    time.sleep(2)

    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser') 

    content_list = soup.find_all("div", "search_result")

    for i in content_list:
        #print(i.text.strip())
        # 번호출력
        no2.append(no)
        print('번호:', no)
        no += 1

        try:
            title = i.find("h3", "dic").get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            title = title.replace("\n", "")
            title = title.replace("\t", "")
            title2.append(title)
            print('제목:' ,title.strip())
        try:
            contents = i.get_text()
        except AttributeError:#예외처리
            continue
        except TypeError:#예외처리
            continue
        else:
            contents = contents.replace("\n", "")
            contents = contents.replace("\t", "")
            contents = contents.replace(title,"")
            contents2.append(contents)
            print('내용:' ,contents.strip())
            print('\n')       
    if not contents2:#아무 검색결과가 없을 때 예외처리
        print("검색결과가 없습니다.")
        restart()
    else:  
        #크롤링 끝
        #driver.quit()#새창 닫고
        driver.close()
        driver.switch_to_window(driver.window_handles[0])#창전환#원래창으로 이동
        #저장메뉴 호출
        output_save()
        
        


#종료함수
def close() :
    print('='*100)
    print("네이버 크롤러를 종료합니다. \n웹 브라우저를 종료합니다.")
    try:
        driver.close()
    except InvalidSessionIdException :
        pass
        
    
    
#재시작 함수()
def restart():      
    save = ['Y', 'y', 'N', 'n']
    ask_save = ""
    while ask_save not in save :
        time.sleep(2)
        ask_save = input('\n다른 단어를 검색해 크롤링을 하시겠습니까?(Y/N) > ')
        if ask_save in save: 
            continue
        else:
            print('Y, y, N, n 중에서 선택해 주세요.')  
    
    if ask_save == 'Y' or ask_save == 'y':
        driver.find_element_by_link_text('NAVER').click()#메인으로 이동
        time.sleep(2)
        start() #크롤러 시작함수 호출
    else:
        close()


#크롤링 뒤 결과 없음 체크 함수
def cheak_contents():
    if not contents2:#아무 검색결과가 없을 때 예외처리
        print("검색결과가 없습니다.")
        restart()
    else:
        #저장메뉴 호출
        output_save()    
        
        
#크롤링 섹션메뉴 선택 함수
def display_menu():
    # 함수 사전 지역변수 
    #0~5까지의 숫자선택이 아니면 반복 선택문 출력
    menus = [0,1,2,3,4,5]
    pick_menu = 6
    while pick_menu not in menus :
        try:
            pick_menu = int(input('메뉴를 선택해 주세요 > '))  
        except ValueError:
            print('보기 숫자 중에서 선택해 주세요.')
        except KeyError:
            print('보기 숫자 중에서 선택해 주세요.')
        else:
            if pick_menu not in menus:
                print('보기 숫자 중에서 선택해 주세요.')
                continue
            global menu
            menu = pick_menu
            
            
#세이브 메뉴 선택 함수
def save_menu():
    # 함수 사전 지역변수 
    #0~3까지의 숫자선택이 아니면 반복 선택문 출력
    menus = [0,1,2,3,4]
    pick_menu = 6
    while pick_menu not in menus :
        try:
            pick_menu = int(input('메뉴를 선택해 주세요 > '))
            if pick_menu not in menus :
                print('보기 숫자 중에서 선택해 주세요.')
        except ValueError:
            print('보기 숫자 중에서 선택해 주세요.')
        else:
            global save_form
            save_form = pick_menu

           
            
#문자열이 빈 문자열이면 False, 아니면 True를 반환하는 함수
def isNotBlank (myString):
    return bool(myString and myString.strip())


#타이틀 쇼잉 및 시작
def show_title_start():
   #타이틀
    print('='*100)
    print('네이버 크롤러 version 1.1')
    print('='*100)
    time.sleep(1)
    open_br()
    
    
#프로그램 구동
show_title_start()


네이버 크롤러 version 1.1
검색어를 입력해 주세요: 크리스피 도넛
[메뉴] 1.블로그  2.뉴스  3.카페  4.지식백과  5.어학사전 (종료. 0)

메뉴를 선택해 주세요 > 1
블로그 섹션의 "크리스피 도넛"(으)로 수집된 데이터를 추출합니다.

